# Notebook for tests and experiments

In [54]:
import pandas as pd
import numpy as np
import re
import pytz
import os
from pathlib import Path
import sys
sys.path.append("/home/jovyan/shared/service-data")

from src.clean import clean_percentage, normalize_string, standardize_column_names, clean_fiscal_yr
from src.load import load_csv_from_raw
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

In [56]:
# Define the base directory
base_dir = Path.cwd()
parent_dir = base_dir.parent
service_inventory = merge_si()

Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported si.csv to /home/jovyan/shared/service-data/outputs
Exported sid_list.csv to /home/jovyan/shared/service-data/outputs/utils


In [134]:
# how many new services and how many services retired?
si = service_inventory.loc[:, ['fiscal_yr', 'service_id', 'org_id']]
si = si[(si['fiscal_yr'] != '2024-2025')]

si_latest = si.loc[si.groupby('service_id')['fiscal_yr'].idxmax()]
si_first = si.loc[si.groupby('service_id')['fiscal_yr'].idxmin()]

si = si.merge(si_latest, on='service_id', suffixes=('', '_latest'))
si = si.merge(si_first, on='service_id', suffixes=('', '_first'))

si_last = si.groupby(['org_id', 'fiscal_yr_latest'])['service_id'].nunique().reset_index()
si_last = si_last.rename(columns={'service_id':'services_reported_for_last_time'})

si_first = si.groupby(['org_id', 'fiscal_yr_first'])['service_id'].nunique().reset_index()
si_first = si_first.rename(columns={'service_id':'services_reported_for_first_time'})


si_first

,org_id,fiscal_yr_first,services_reported_for_first_time
0,1,2018-2019,30
1,1,2019-2020,5
2,1,2020-2021,6
3,1,2021-2022,4
4,1,2022-2023,5
...,...,...,...
223,95,2023-2024,3
224,98,2018-2019,3
225,98,2019-2020,2
226,99,2018-2019,5


In [164]:
si = service_inventory.loc[:,[
        'service_id', 
        'service_name_en', 
        'service_name_fr', 
        'fiscal_yr', 
        'department_en', 
        'department_fr', 
        'org_id'
    ]
]

si_latest = si.loc[si.groupby('service_id')['fiscal_yr'].idxmax(), ['service_id', 'fiscal_yr']]
si_first = si.loc[si.groupby('service_id')['fiscal_yr'].idxmin(), ['service_id', 'fiscal_yr']]

si = si.merge(si_first, on='service_id', suffixes=('', '_first'))
si = si.merge(si_latest, on='service_id', suffixes=('', '_latest'))


si = si[si['fiscal_yr'] == si['fiscal_yr_latest']].drop(columns='fiscal_yr')

si

,service_id,service_name_en,service_name_fr,department_en,department_fr,org_id,fiscal_yr_first,fiscal_yr_latest
0,1000,Reconciliation,Réconciliation,Crown-Indigenous Relations and Northern Affair...,Relations Couronne-Autochtones et Affaires du ...,129,2018-2019,2018-2019
46,1010,Fisheries and Aquaculture Clean Technology Ado...,Programme d'adoption des technologies propres ...,Fisheries and Oceans Canada,Pêches et Océans Canada,125,2018-2019,2022-2023
66,1014,National Indigenous Representative Organizations,Organisations autochtones représentatives nati...,Crown-Indigenous Relations and Northern Affair...,Relations Couronne-Autochtones et Affaires du ...,129,2018-2019,2018-2019
67,1015,MPA Activity Plan Application Process - Anguni...,Processus de demande d'activités pour la ZPM -...,Fisheries and Oceans Canada,Pêches et Océans Canada,125,2018-2019,2022-2023
72,1016,MPA Activity Plan Application Process - SGaan ...,Processus de demande d'activités pour la ZPM -...,Fisheries and Oceans Canada,Pêches et Océans Canada,125,2018-2019,2022-2023
...,...,...,...,...,...,...,...,...
8925,1273,War Veterans Allowance,Allocation aux anciens combattants,Veterans Affairs Canada,Anciens Combattants Canada,139,2018-2019,2023-2024
8926,53,Review and Appeal hearings,Audiences de révision et d'appel,Veterans Review and Appeal Board,Tribunal des anciens combattants (révision et ...,333,2018-2019,2023-2024
8927,2104,Women's Program,Programme de promotion de la femme,Women and Gender Equality Canada,Femmes et Égalité des genres Canada,246,2019-2020,2023-2024
8928,2105,Gender-Based Violence Program,Programme de financement de la lutte contre la...,Women and Gender Equality Canada,Femmes et Égalité des genres Canada,246,2019-2020,2023-2024
